# Detection of suspicious objects using YOLOv5.
We have selected 6 classes of suspicious objects:

* Person
* Knife
* LifeJacket
* Bag
* Baseball bat
* Gun

### Link to download the data: https://drive.google.com/file/d/1dI0iC7lwdoStuwDSGtx-yDRr1xfYOG4c/view?usp=sharing
The data is already divided into training, validation, and test sets.

### Once the data is downloaded, create a folder named "datasets" at the root of the project and place the data in this folder.


#### Datasets directory structure: 

<p align="center">
  <img src="datasets_folder.png">
</p>


Clone Yolov5

In [1]:
clone = False
if clone:
    !git clone --depth 1 https://github.com/ultralytics/yolov5
    #!rsync -av yolov5/ . 
    !xcopy yolov5\* /s /e .
    !rmdir /s /q yolov5

Install Dependencies

In [2]:
if clone:
    !pip install -r requirements.txt

In [3]:
import os
import torch
import utils
import pandas as pd 
from tqdm import tqdm
from labels_creation import *
from sklearn.model_selection import train_test_split
from IPython.display import Image, clear_output  # to display images
display = utils.notebook_init()  # checks

YOLOv5  2023-5-4 Python-3.7.9 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)


Setup complete  (16 CPUs, 31.7 GB RAM, 596.8/772.8 GB disk)


In [4]:
torch.cuda.is_available()

True

In [5]:
SEED = 42
BD_dir ='BD_suspect'
SPLIT_TRAIN_VAL= False 
VAL_SIZE = 0.3

Split data into train_data and val_data

In [6]:
def split_img_label(data_train, data_val, BD_dir):
    
    train_ind=list(data_train.index)
    val_ind=list(data_val.index)
    
    # ------------------------- Train folder -----------------------------
    # Empty train_img folder
    img_train_path = "datasets/BD_suspect/images/train/"
    img_files = os.listdir(img_train_path)
    for f in img_files:
        if os.path.exists(img_train_path + f):
            pass
            os.remove(img_train_path + f)
        
    # Empty train_lbl folder
    lbl_train_path = "../datasets/BD_suspect/labels/train/"
    lbl_train_files = os.listdir(lbl_train_path)
    for f in lbl_train_files:
        if os.path.exists(lbl_train_path + f):
            pass
            os.remove(lbl_train_path + f)
    
    for i in tqdm(range(len(train_ind))):
        #images
        train_img_src = data_train[train_ind[i]].replace('/','\\')
        train_img_dest = "..\datasets\\" + BD_dir + "\images\\train\\"+data_train[train_ind[i]].split("/")[-1]


        #labels
        train_lbl_src = (data_train[train_ind[i]].split('.jpg')[0]+'.txt').replace('images','labels')
        train_lbl_src = train_lbl_src.replace('/','\\')
        train_lbl_dest = "..\datasets\\" + BD_dir + "\labels\\train\\" + data_train[train_ind[i]].split("/")[-1].split('.jpg')[0]+'.txt'

        #copy data
        os.system('copy ' + train_img_src + ' ' + train_img_dest)
        os.system('copy ' + train_lbl_src + ' ' + train_lbl_dest)

    # ---------------------- Val folder ----------------------------------
    # Empty val_img folder
    img_val_path = "../datasets/BD_suspect/images/val/"
    img_val_files = os.listdir(img_train_path)
    for f in img_val_files:
        if os.path.exists(img_val_path + f):
            pass
            os.remove(img_val_path + f)
        
    # Empty val_lbl folder
    lbl_val_path = "../datasets/BD_suspect/labels/val/"
    lbl_val_files = os.listdir(lbl_train_path)
    for f in lbl_val_files:
        if os.path.exists(lbl_val_path + f):
            pass
            os.remove(lbl_val_path + f)

    for i in tqdm(range(len(val_ind))):
        #images
        val_img_src = data_val[val_ind[i]].replace('/','\\')
        val_img_dest = "..\datasets\\" + BD_dir + "\images\\val\\"+data_val[val_ind[i]].split("/")[-1]

        #labels
        val_lbl_src = (data_val[val_ind[i]].split('.jpg')[0]+'.txt').replace('images','labels')
        val_lbl_src = val_lbl_src.replace('/','\\')
        val_lbl_dest = "..\datasets\\" + BD_dir + "\labels\\val\\" + data_val[val_ind[i]].split("/")[-1].split('.jpg')[0]+'.txt'

        #copy data
        os.system('copy ' + val_img_src + ' ' + val_img_dest)
        os.system('copy ' + val_lbl_src + ' ' + val_lbl_dest)
        

In [7]:
if SPLIT_TRAIN_VAL:
    PATH = 'datasets/' + BD_dir + '/images/data_train/'
    list_img=[img for img in os.listdir(PATH) if img.endswith('.jpg')==True]
    path_img=[]

    for i in range (len(list_img)):
        path_img.append(PATH+list_img[i])
    
    df=pd.DataFrame(path_img)

    # split data
    data_train, data_val, _, _ = train_test_split(df[0], df.index, test_size=VAL_SIZE, random_state=SEED)

    split_img_label(data_train, data_val, BD_dir)

Model configuration

In [2]:
model_to_be_trained = "yolov5n"
nbr_epochs = 100
img_size = 640 
batch_size =  16
load_weights = True 

In [12]:
init_train = False
continue_train = True

Initial training

In [10]:
if init_train:
    !python train.py --img "$img_size"  --batch "$batch_size" --epochs "$nbr_epochs" --data "BD_suspect.yaml"  --weights yolov5n.pt --name "suspect_results_" --project "HackAI_OD_newDB" --seed 42 --cache disk

Continue Training

In [13]:
if continue_train:
    !python train.py --img "$img_size"  --batch "$batch_size" --epochs 1  --data "BD_suspect.yaml"  --weights "HackAI_OD_newDB\suspect_results_1\weights\best.pt" --name "suspect_results_" --project "HackAI_OD_newDB" --seed 42 --cache disk

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
requirements: F:\VSCProjects\GitHubYolov5\requirements.txt not found, check failed.


wandb: WARNING  wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: tojov (tojov-a). Use `wandb login --relogin` to force relogin
train: weights=HackAI_OD_newDB\suspect_results_1\weights\best.pt, cfg=, data=BD_suspect.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=1, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=disk, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=HackAI_OD_newDB, name=suspect_results_, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=42, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  2023-5-4 Python-3.7.9 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384Mi

Evaluation on the validation set

In [14]:
!python val.py --img "$img_size" --weights  "HackAI_OD_newDB\suspect_results_2\weights\best.pt" --data "BD_suspect.yaml" 

requirements: F:\VSCProjects\GitHubYolov5\requirements.txt not found, check failed.

val: data=BD_suspect.yaml, weights=['HackAI_OD_newDB\\suspect_results_2\\weights\\best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2023-5-4 Python-3.7.9 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)

Fusing layers... 
Model summary: 157 layers, 1767283 parameters, 0 gradients, 4.2 GFLOPs

val: Scanning F:\VSCProjects\GitHubYolov5\datasets\BD_suspect\labels\val...:   0%|          | 0/1630 [00:00<?, ?it/s]
val: Scanning F:\VSCProjects\GitHubYolov5\datasets\BD_suspect\labels\val... 1 images, 0 backgrounds, 0 corrupt:   0%|          | 1/1630 [00:04<2:01:10,  4.46s/it]
val: Scanning F:\VSCProjects\GitHubYolov5\datasets\BD_suspect\labels\val... 97 images, 0 backgrounds, 0 corrupt:   6%|▌         | 97/1630 [00:04<00

Evaluation on the test set

In [5]:
!python val.py --img "$img_size" --weights  "HackAI_OD_newDB\suspect_results_2\weights\best.pt" --data "BD_suspect.yaml" --task test

requirements: F:\VSCProjects\OD_Yolov5\requirements.txt not found, check failed.


val: data=BD_suspect.yaml, weights=['HackAI_OD_newDB\\suspect_results_2\\weights\\best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2023-5-4 Python-3.7.9 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)

Fusing layers... 
Model summary: 157 layers, 1767283 parameters, 0 gradients, 4.2 GFLOPs

test: Scanning F:\VSCProjects\OD_Yolov5\datasets\BD_suspect\labels\test...:   0%|          | 0/2276 [00:00<?, ?it/s]
test: Scanning F:\VSCProjects\OD_Yolov5\datasets\BD_suspect\labels\test... 1 images, 0 backgrounds, 0 corrupt:   0%|          | 1/2276 [00:04<2:33:16,  4.04s/it]
test: Scanning F:\VSCProjects\OD_Yolov5\datasets\BD_suspect\labels\test... 378 images, 0 backgrounds, 0 corrupt:  17%|█▋        | 378/2276 [00:04<00

Test on one image

In [4]:
!python detect.py --img "$img_size" --weights  "HackAI_OD_newDB\suspect_results_2\weights\best.pt" --conf 0.3 --source "../datasets/BD_suspect/Image_test.png

requirements: F:\VSCProjects\OD_Yolov5\requirements.txt not found, check failed.


detect: weights=['HackAI_OD_newDB\\suspect_results_2\\weights\\best.pt'], source=../datasets/BD_suspect/Image_test.png, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2023-5-4 Python-3.7.9 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)

Fusing layers... 
Model summary: 157 layers, 1767283 parameters, 0 gradients, 4.2 GFLOPs
image 1/1 F:\VSCProjects\OD_Yolov5\datasets\BD_suspect\Image_test.png: 416x640 3 Knifes, 11.0ms
Speed: 0.0ms pre-process, 11.0ms inference, 4.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp


Test on camera

In [18]:
!python detect.py --img "$img_size" --weights  "HackAI_OD_newDB\suspect_results_2\weights\best.pt" --conf 0.3 --source 0

detect: weights=['HackAI_OD_newDB\\suspect_results_2\\weights\\best.pt'], source=0, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2023-5-4 Python-3.7.9 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)

Fusing layers... 
Model summary: 157 layers, 1767283 parameters, 0 gradients, 4.2 GFLOPs
1/1: 0...  Success (inf frames 640x480 at 30.00 FPS)

0: 480x640 1 Person, 218.5ms
0: 480x640 1 Person, 7.0ms
0: 480x640 1 Person, 6.0ms
0: 480x640 1 Person, 9.0ms
0: 480x640 1 Person, 5.0ms
0: 480x640 1 Person, 5.0ms
0: 480x640 1 Person, 7.0ms
0: 480x640 1 Person, 5.0ms
0: 480x640 1 Person, 6.0ms
0: 480x640 1 Person, 5.0ms
0: 

requirements: F:\VSCProjects\GitHubYolov5\requirements.txt not found, check failed.
